<a href="https://colab.research.google.com/github/andrecsq/CS/blob/master/fb_quantic_budget_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Budget Manager - Configuração de Orçamento

O objetivo desta etapa é criar um dataset de configuração de orçamento das pools e o orçamento dos objetivos.

### Configuração do notebook

Utilizaremos as bibliotecas pandas e numpy para trabalhar com o dataset. Para o carregamento e consulta dos dados, utilizaremos a biblioteca do google cloud. Abaixo, iremos configurar os parâmetros globais do algoritmo.

In [79]:
import os
import warnings
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from google.cloud import bigquery
from google.cloud.bigquery import magics
from google.oauth2 import service_account

warnings.filterwarnings('ignore', category=FutureWarning)

In [80]:
PARTITION_DATE = locals().get('yesterday_ds', (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d"))
PARTITION_DATE_NODASH = locals().get('yesterday_ds_nodash', (datetime.today() - timedelta(days=1)).strftime("%Y%m%d"))

GOOGLE_AUTH_CREDENTIALS_FILE = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', '/content/credentials.json')

BIGQUERY_PROJECT = 'pareto-bi'
BIGQUERY_DATASET = locals().get('gbq_dataset', 'BUDGET_MANAGER_TEST')

BIGQUERY_BUDGET_CONFIG_TABLE_PART = f'{BIGQUERY_PROJECT}.{BIGQUERY_DATASET}.fb_quantic_budget_config${PARTITION_DATE_NODASH}'
BIGQUERY_BUDGET_CONFIG_TABLE = f'{BIGQUERY_PROJECT}.{BIGQUERY_DATASET}.fb_quantic_budget_config'
BIGQUERY_OPT_IN_TABLE = f'{BIGQUERY_PROJECT}.BUDGET_MANAGER.fb_manual_config_opt_in_native'

In [81]:
credentials = service_account.Credentials.from_service_account_file(GOOGLE_AUTH_CREDENTIALS_FILE)
magics.context.credentials = credentials

client = bigquery.Client(credentials=credentials, project=BIGQUERY_PROJECT)

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Montagem do dataset

In [82]:
job_config=bigquery.QueryJobConfig(
  destination=BIGQUERY_BUDGET_CONFIG_TABLE_PART,
  write_disposition="WRITE_TRUNCATE",
  create_disposition="CREATE_IF_NEEDED",
  schema_update_options=["ALLOW_FIELD_ADDITION", "ALLOW_FIELD_RELAXATION"],
  use_legacy_sql=False,
  time_partitioning=bigquery.table.TimePartitioning(field="day"),
)

In [83]:
query = client.query(f"""

WITH dates AS (
  SELECT DATE_SUB(CURRENT_DATE('America/Sao_Paulo'), INTERVAL 1 DAY) AS yesterday
),

active_campaigns_with_budget AS (
    SELECT DISTINCT
        account_id,
        account_name,
        campaign_id,
        campaign_name,
        IF(adset_bid_strategy IS NOT NULL, FALSE, TRUE) AS is_cbo,
        adgroup_id,
        adgroup_name,
        adset_optimization_goal,
        REPLACE(JSON_EXTRACT(adset_promoted_object, '$.custom_event_type'), '\"', '') AS custom_event_type
    FROM `pareto-bi.ETL_RAW.fb_adgroup`, dates
    WHERE 
        day = yesterday
        AND campaign_effective_status  = 'ACTIVE'
        AND adset_effective_status = 'ACTIVE'
),

calculating_objective_and_result AS (
    SELECT DISTINCT
        account_id,
        account_name,
        campaign_id,
        campaign_name,
        is_cbo,
        adgroup_id,
        adgroup_name,
        UTIL.get_budget_manager_objective_fb(adset_optimization_goal, custom_event_type) AS objective_name,
        UTIL.get_budget_manager_kpi_fb(adset_optimization_goal, custom_event_type) AS objective_kpi,
    FROM active_campaigns_with_budget
),

opt_in_elements AS (
    SELECT 
        TRIM(cluster_name) AS cluster_name,
        SAFE_CAST(cluster_budget AS FLOAT64) AS cluster_budget,
        TRIM(status) AS automation_status,
        SAFE_CAST(TRIM(account_id) AS INT64) AS account_id, 
        SAFE_CAST(TRIM(campaign_id) AS INT64) AS campaign_id,
        CAST(NULL AS INT64) AS adgroup_id
    FROM 
        {BIGQUERY_OPT_IN_TABLE} AS t_optin, 
        UNNEST(SPLIT(t_optin.campaigns_ids, ",")) AS campaign_id
    UNION ALL
    SELECT 
        TRIM(cluster_name) AS cluster_name,
        SAFE_CAST(cluster_budget AS FLOAT64) AS cluster_budget,
        TRIM(status) AS automation_status,
        SAFE_CAST(TRIM(account_id) AS INT64) AS account_id, 
        CAST(NULL AS INT64) AS campaign_id,
        SAFE_CAST(TRIM(adset_id) AS INT64) AS adgroup_id
    FROM 
        {BIGQUERY_OPT_IN_TABLE} AS t_optin, 
        UNNEST(SPLIT(t_optin.adsets_ids, ",")) AS adset_id
),

opt_in_applied AS (
    SELECT  
        opt_in_elements.cluster_name,
        opt_in_elements.cluster_budget,
        opt_in_elements.automation_status,
        t_accounts.*  
    FROM calculating_objective_and_result AS t_accounts
    LEFT JOIN opt_in_elements USING (account_id, campaign_id)
    UNION ALL    
    SELECT  
        opt_in_elements.cluster_name,
        opt_in_elements.cluster_budget,
        opt_in_elements.automation_status,
        t_accounts.* 
    FROM calculating_objective_and_result AS t_accounts
    LEFT JOIN opt_in_elements USING (account_id, adgroup_id)
),

ranking_objective_kpis AS (
    SELECT
        *,
        UTIL.get_budget_manager_kpi_rank_fb(objective_name, objective_kpi) AS kpi_rank
    FROM opt_in_applied
),

getting_best_kpi_and_result AS (
    SELECT 
        *,
        FIRST_VALUE(objective_kpi) OVER (PARTITION BY account_id, objective_name ORDER BY kpi_rank) AS first_kpi, -- CPR
        FIRST_VALUE(kpi_rank) OVER (PARTITION BY account_id, objective_name ORDER BY kpi_rank) AS first_kpi_rank, -- 2   
    FROM ranking_objective_kpis     
),

ensuring_one_kpi_per_account_and_objective AS (
    SELECT 
        * EXCEPT (objective_kpi, kpi_rank, first_kpi, first_kpi_rank),
        IF(first_kpi_rank = kpi_rank, objective_kpi, first_kpi) AS objective_kpi,
        COUNT(DISTINCT adgroup_id) OVER (PARTITION BY account_id, cluster_name, objective_name) AS quant_adsets_objective,
        COUNT(DISTINCT adgroup_id) OVER (PARTITION BY account_id, cluster_name) AS quant_adsets_cluster
    FROM getting_best_kpi_and_result     
),

getting_quantic_budget AS (
    SELECT
        accounts.id AS account_id,
        IF(accounts.budget_goal = 0, NULL, accounts.budget_goal) AS account_budget,
        ROW_NUMBER() OVER (PARTITION BY  accounts.id ORDER BY service_type) AS row_n
    FROM `pareto-bi.QUANTIC.quantic_base_array`  AS t_base
    CROSS JOIN UNNEST (fb_accounts) AS accounts
    WHERE status = 'active'
),

bringing_only_one_budget_per_account_id AS (
    SELECT * EXCEPT (row_n)
    FROM getting_quantic_budget
    WHERE row_n = 1
),

past_investment AS (
    SELECT 
        account_id,
        IFNULL(SUM(cost), 0) AS last_month_investment
    FROM `pareto-bi.ETL_RAW.fb_adgroup`
    WHERE 
        DATE_TRUNC(day, MONTH) = 
        DATE_TRUNC(DATE_SUB(CURRENT_DATE('America/Sao_Paulo'), INTERVAL 1 MONTH), MONTH)
    GROUP BY 1
),

ensuring_all_accounts_have_budget AS (
    SELECT DISTINCT
        DATE('{PARTITION_DATE}') AS day,
        CAST(NULL AS INT64) AS pool_id,
        IFNULL(cluster_name, "Manual Cluster") AS pool_name,
        cluster_budget AS pool_budget,
        "pareto_account_budget" AS pool_budget_source,
        0 AS version,
        IFNULL(automation_status, 'DISABLED') AS automation_status,
        account_id,
        account_name,
        ROUND(IFNULL(t_budget.account_budget, IFNULL(last_month_investment, 0))) AS account_budget,
        objective_name,
        objective_kpi,
        SAFE_DIVIDE(quant_adsets_objective, quant_adsets_cluster) AS percentage_of_objective,
        campaign_id,
        campaign_name,
        is_cbo,
        adgroup_id,
        adgroup_name,
    FROM ensuring_one_kpi_per_account_and_objective
    LEFT JOIN bringing_only_one_budget_per_account_id AS t_budget USING (account_id)
    LEFT JOIN past_investment USING (account_id)
),

ensuring_all_clusters_have_budget AS (
    SELECT
        day,
        pool_id,
        pool_name,
        ROUND(IFNULL(pool_budget, account_budget), 2) AS pool_budget,
        pool_budget_source,
        version,
        automation_status,
        account_id,
        account_name,
        account_budget,
        objective_name,
        objective_kpi,
        ROUND(IFNULL(pool_budget, account_budget)*percentage_of_objective, 2) AS objective_budget,
        ROUND(percentage_of_objective, 2) AS percentage_of_objective,
        campaign_id,
        campaign_name,
        is_cbo,
        adgroup_id,
        adgroup_name,
        IF(is_cbo, campaign_id, adgroup_id) AS object_id,
        IF(is_cbo, campaign_name, adgroup_name) AS object_name,
        IF(is_cbo, 'campaign', 'adset') AS object_type,
        CAST(NULL AS BOOL) is_pool_modified
    FROM ensuring_all_accounts_have_budget
)

SELECT *
FROM ensuring_all_clusters_have_budget

""", job_config=job_config)

In [84]:
query.result()

## Adição de `is_pool_modified`



In [85]:
query = client.query(f"""

WITH current_config AS (
    SELECT *
    FROM {BIGQUERY_BUDGET_CONFIG_TABLE}
    WHERE day = DATE("{PARTITION_DATE}")
),

past_configs AS (
    SELECT *
    FROM {BIGQUERY_BUDGET_CONFIG_TABLE}
    WHERE day < DATE("{PARTITION_DATE}")
),

last_config AS (
    SELECT *
    FROM past_configs
    WHERE day = (SELECT MAX(day) FROM past_configs)
)

-- Decidimos que todos os casos que devemos criar card é quando HOJE o budget está configurado na planilha.
-- Para a configuração passada, não importa se estava configurada ou não.

SELECT 
    current_config.* EXCEPT(is_pool_modified),
    IF(
        IFNULL(current_config.pool_budget, 0) != IFNULL(last_config.pool_budget, 0)
        AND current_config.automation_status = 'ENABLED', 
        TRUE, 
        FALSE
    ) AS is_pool_modified
FROM current_config
LEFT JOIN last_config USING(pool_name, account_id, campaign_id, adgroup_id)

""", job_config=job_config)

In [86]:
query.result()